In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import  preprocessing
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold,StratifiedKFold
#import matplotlib.pyplot as plt

/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from mochi import *
import pickle

In [5]:
from sklearn.ensemble import RandomForestClassifier as RFC

In [9]:
from sklearn.ensemble import ExtraTreesClassifier as ETC

In [3]:
data_path = "/home/raku/kaggleData/2sigma/xgb142/"
train_file = data_path + "xgb1.42-train.json"
test_file = data_path + "xgb1.42-test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)

pickl_file = data_path+'xgb142features.pickle'
fileObject = open(pickl_file,'r') 
features=pickle.load(fileObject)   
fileObject.close()
print len(features)

(49352, 293)
(74659, 292)
220


In [4]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 2333)

In [10]:
cv_scores=[]
i=0
for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
       #filter the features
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    et = ETC(100)
    et.fit(dev_X,dev_y)
    preds = rf.predict_proba(val_X)

    loss = log_loss(val_y, preds)
    
    cv_scores.append(loss)
    i+=1
    print 'loss for the turn '+str(i)+' is '+str(loss)
    break

loss for the turn 1 is 0.585039020078


In [ ]:
print 'The mean of the cv_scores is:'
print np.mean(cv_scores)

cvResult = CVstatistics(cv_result,'mlogloss')

meanTestError = cvResult.result.filter(like='test').mean(axis=1)

print meanTestError[meanTestError==np.min(meanTestError)]